In [1]:
# To make sure that this notebook is being run on a Spark 2.0 cluster, let's see if we can access the SparkSession - the new entry point of Apache Spark 2.0.
# If this fails, then you are not connected to a Spark 2.0 cluster. Please recreate your cluster and select the version to be "Spark 2.0 (Scala 2.10)".
spark

In [23]:
from pyspark.sql import Row
jstr1 = u'{"header":{"id":12345,"foo":"bar"},"body":{"id":111000,"name":"foobar","sub_json":{"id":54321,"sub_sub_json":{"col1":20,"col2":"somethong"}}}}'
jstr2 = u'{"header":{"id":12346,"foo":"baz"},"body":{"id":111002,"name":"barfoo","sub_json":{"id":23456,"sub_sub_json":{"col1":30,"col2":"something else"}}}}'
jstr3 = u'{"header":{"id":43256,"foo":"foobaz"},"body":{"id":20192,"name":"bazbar","sub_json":{"id":39283,"sub_sub_json":{"col1":50,"col2":"another thing"}}}}'
df = spark.createDataFrame([Row(json=jstr1),Row(json=jstr2),Row(json=jstr3)])
#df.collect()
df.printSchema()

root
 |-- json: string (nullable = true)



In [6]:
new_df = spark.read.json(df.rdd.map(lambda r: r.json))
#new_df.collect()
new_df.printSchema()

root
 |-- body: struct (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- sub_json: struct (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- sub_sub_json: struct (nullable = true)
 |    |    |    |-- col1: long (nullable = true)
 |    |    |    |-- col2: string (nullable = true)
 |-- header: struct (nullable = true)
 |    |-- foo: string (nullable = true)
 |    |-- id: long (nullable = true)



In [56]:
from pyspark.sql.functions import *
jsonSchema = StructType([ StructField("last_reported", TimestampType(), True), StructField("name", StringType(), True) ])
strm = df.select(from_json(df.json, jsonSchema).alias("foo"))#.rdd.flatMap(lambda x: x).
#strm.select(explode("foo"))
map{case Row(m1: String, m2: String, m3: String, f1: String, f2: String) =>
  (m1, m2, m3, f1, f2)}

aRDD = df.withColumn("moreData", explode($"moreData"))
  .select(columns: _*)
  .rdd
#df.select(explode(split(col("word"), "\s+")).alias("word")).show()

#strm.rdd.flatMap(lambda x: x).collect()
#strm = df.select(map(lambda r: r.json))
#strm.printSchema()

AnalysisException: "cannot resolve 'explode(`foo`)' due to data type mismatch: input to function explode should be array or map type, not StructType(StructField(last_reported,TimestampType,true), StructField(name,StringType,true));;\n'Project [explode(foo#139) AS List()]\n+- Project [jsontostruct(StructField(last_reported,TimestampType,true), StructField(name,StringType,true), json#57) AS foo#139]\n   +- LogicalRDD [json#57]\n"

In [47]:
from pyspark.sql.types import *
import json
from pyspark.sql.functions import *
# Since we know the data format already, let's define the schema to speed up processing (no need for Spark to infer schema)
#jsonSchema = StructType([ StructField("last_reported", TimestampType(), True), StructField("name", StringType(), True) ])

jsonSchema = StructType([StructField("name", StringType(), True) ])

# Strem DataFrame representing data in the JSON files
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9985) \
    .schema(jsonSchema) \
    .load()

lines.isStreaming
#words = lines.select(explode(split(lines.value, ":")).alias("word"))
#stream_objects = lines.select(json.dumps(lines.value))
#type(stream_objects)

#stream_objects = lines.select(from_json(lines.value, jsonSchema).alias("stream_object"))
#stream_objects = lines.select(from_json(lines.value, jsonSchema).alias("stream_object"))
#stream_objects = spark.read.json(lines.rdd.map(lambda r: r.json))
#new_df.collect()
#stream_objects.printSchema()
strm = df.select(from_json(df.json, jsonSchema))#.rdd.flatMap(lambda x: x).
#strm.rdd.flatMap(lambda x: x).collect()
strm.isStreaming
#stream_objects.isStreaming

#wordCounts = words.groupBy("word").count()

False

In [ ]:
#streamingCountsDF = (                 
#  stream_objects
#    .groupBy(
#      stream_objects.stream_object, 
#      window(stream_objects.stream_object, "1 hour"))
#    .count()
#    )
#streamingCountsDF.isStreaming


In [3]:
spark.conf.set("spark.sql.shuffle.partitions", "2")  # keep the size of shuffles small

query = (
#  streamingCountsDF
    stream_objects
    .writeStream
    .format("console")        # memory = store in-memory table (for testing only in Spark 2.0)
#    .queryName("counts")     # counts = name of the in-memory table
    .outputMode("append")  # complete = all the counts should be in the table
    .start()
)

In [5]:
query.awaitTermination()

KeyboardInterrupt: 